In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import combinations

In [9]:
d= np.loadtxt('prices.txt', dtype=float)
d= d.T
d

array([[38.25, 38.11, 37.9 , ..., 36.58, 36.43, 36.24],
       [69.06, 69.98, 69.56, ..., 50.8 , 51.19, 50.64],
       [62.08, 62.39, 62.82, ..., 44.25, 44.02, 43.43],
       ...,
       [24.02, 24.04, 23.76, ..., 15.81, 15.75, 15.74],
       [76.03, 76.05, 75.66, ..., 80.93, 80.8 , 80.5 ],
       [27.5 , 27.54, 27.55, ..., 31.83, 31.78, 31.59]])

In [11]:
returns = np.diff(np.log(d),axis = 1)

In [13]:
selected_indices = [18, 7, 38]
selected_returns = returns[selected_indices, :]
cor_matrix_3 = np.corrcoef(selected_returns)
cor_matrix_3

array([[1.        , 0.05073346, 0.12795815],
       [0.05073346, 1.        , 0.11477601],
       [0.12795815, 0.11477601, 1.        ]])

In [15]:
np.linalg.det(cor_matrix_3)

0.9693694927164949

In [17]:
cor_matrix = np.corrcoef(returns)

In [ ]:
cor_matrix

In [ ]:
def find_top_10_asset_triplets(cor_matrix, top_n=10):
    n = cor_matrix.shape[0]
    triplet_scores = []

    # Iterate over all possible 3-asset combinations
    for triplet in combinations(range(n), 3):
        sub_corr = cor_matrix[np.ix_(triplet, triplet)]
        off_diag_sum = np.sum(np.abs(sub_corr)) - np.sum(np.abs(np.diag(sub_corr)))
        triplet_scores.append((triplet, off_diag_sum))

    # Sort triplets by descending off-diagonal sum
    triplet_scores.sort(key=lambda x: x[1], reverse=True)

    # Return top_n triplets and their scores
    return triplet_scores[:top_n]


top_trips=find_top_10_asset_triplets(cor_matrix)
top_trips

In [ ]:
def compute_spread_zscore_3assets(p1, p2, p3):
    # Regress p1 on p2 and p3: p1 = beta1 * p2 + beta2 * p3 + error
    X = np.vstack([p2, p3]).T  # shape (n_samples, 2)
    y = p1
    beta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    spread = y - X.dot(beta)
    z = (spread - np.mean(spread)) / np.std(spread)
    return spread, z, beta

def simulate_trades_3assets(z, entry=1.0, exit=0.0):
    positions = np.zeros(len(z))
    for t in range(1, len(z)):
        if z[t-1] < -entry:
            positions[t] = 1  # Long spread
        elif z[t-1] > entry:
            positions[t] = -1  # Short spread
        elif abs(z[t-1]) < exit:
            positions[t] = 0  # Exit position
        else:
            positions[t] = positions[t-1]  # Hold previous position
    return positions

def compute_pnl_3assets(positions, spread):
    spread_diff = np.diff(spread)
    pnl = positions[:-1] * spread_diff  # lag position by 1
    return np.cumsum(pnl)